In [13]:
# Project description
#
# The aim of this notebook is to explore and find ways to predict heart disease with as high performance as possible
# using the "UCI Heart Disease" dataset, extracted from Kaggle.com.
# CRISP-DM?????
# -------------------------

# Imports
import pandas as pd
import numpy as np

data = pd.read_csv('dataset.csv', sep = ',')

# checking columns of dataset
list(data.columns)
# checking the first few rows to get a feel for the contents of the dataset 
data.head()
data.describe()
# checking distribution between positive and negative subjects in the dataset
data.target.value_counts()
# checking for duplicate rows
data.duplicated().value_counts()
# one duplicate row found, drop it
data = data.drop_duplicates()
# no null cells for any feature
data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf

# Split the data into training and testing sets
x_train, x_test, y_train, y_test  = train_test_split(data.drop('target', 1), data['target'], test_size = 0.25)

model = rf(n_estimators=50)
model = model.fit(x_train, y_train)
classes = model.predict(x_test)
(classes == y_test).sum()/y_test.size


0.8289473684210527